In [54]:
import os
import fitz
from tqdm import tqdm

In [55]:
def check_rate_function(rate_1, rate_2, possible_rate):
        # 檢查兩種方法提取的評價是否相同
    if rate_1 == rate_2 and rate_1!= None:
        return rate_1
    elif rate_1 in possible_rate:
        return rate_1
    elif rate_2 in possible_rate:
        return rate_2
    else :
        return 'NULL'
            

In [56]:
zip_path = r'..\file\hsbc'
count_true = 0
possible_rate = ['MAINTAIN REDUCE', 'MAINTAIN BUY', 'UPGRADE TO HOLD', 'REDUCE', 'MAINTAIN HOLD', 'INITIATE AT REDUCE']
for directory in tqdm(os.listdir(zip_path)):
    directory_path = os.path.join(zip_path, directory)
    rate_1 = None
    rate_2 = None
    rate = None
    try:
        with fitz.open(directory_path) as doc:
            mat = fitz.Matrix(1, 1)
            page = doc.load_page(0)
            rect = page.rect
            page_check_source = doc.load_page(-1)
            text_check_source = page_check_source.get_text()
            page_check_source_2 = doc.load_page(-2)
            text_check_source_2 = page_check_source_2.get_text()
            if 'Hongkong and Shanghai Banking Corporation' in text_check_source or 'Hongkong and Shanghai Banking Corporation' in text_check_source_2 :
                # 檢查版本                
                clip_check_report = fitz.Rect(350, 0, rect.width, 250)
                text_check_report = page.get_text(clip=clip_check_report)
                if any(keyword in text_check_report for keyword in ['Equities']):
                    # 提取評價的第一種方法
                    try:
                        clip_old_report_1 = fitz.Rect(375, 0, rect.width, 500)
                        text_old_report_1 = page.get_text(clip=clip_old_report_1, sort=True).strip()
                        text_old_report_1 = text_old_report_1.split('TARGET')[0].strip()
                        rate_1 = text_old_report_1.split('\n')[-1].strip()
                    except:
                        rate_1 = None
                        # 提取評價的第二種方法
                    try:
                        clip_old_report_2 = fitz.Rect(370, 190, rect.width, 230)
                        rate_2 = page.get_text(clip=clip_old_report_2).strip()
                    except:
                        rate_2 = None
                # else :
                #     print(f'{text_check_report}, {directory_path} NULL')
                rate = check_rate_function(rate_1, rate_2, possible_rate)
            else:
                # False
                print(f'{directory_path} is not posted by hsbc')
    except:
        print(directory_path, 'fail')
    if rate == 'NULL' :
        print('case1', directory_path, rate,
                f'rate_1={rate_1}, rate_2={rate_2}')
    elif rate == 'wrong':
        count_true += 1
        # print(directory_path, rate,
        #       f'text_check_new_report={text_check_new_report}, text_check_old_fast_report={text_check_old_fast_report}')
    elif rate != None :
        count_true += 1
        # print(directory_path, rate,
        #         f'rate_1={rate_1}, rate_2={rate_2}')
    else :
        print('case2', directory_path, rate,
                f'rate_1={rate_1}, rate_2={rate_2}')
# 計算成功和失敗的數量並打印
count_false = len(os.listdir(zip_path)) - count_true
success_rate = round((count_true/len(os.listdir(zip_path))), 3)*100
print(f'共{len(os.listdir(zip_path))}件, 成功{count_true}件, 失敗{count_false}件, 成功率:{success_rate}%')

100%|██████████| 20/20 [00:01<00:00, 12.74it/s]

共20件, 成功20件, 失敗0件, 成功率:100.0%


In [1]:
import fitz
doc = fitz.open(r'..\file\GS\2317_鴻海_高盛_Buy.pdf')
mat = fitz.Matrix(1, 1)
page = doc.load_page(0)  # loads page number 'pno' of the document (0-based)
rect = page.rect
pix = page.get_pixmap(matrix=mat, alpha=False)
pix.save("page-%i.png" % page.number)

In [2]:
# 檢查是否由第一金投顧出版
page_check_source = doc.load_page(-2)
text_check_source = page_check_source.get_text()
if 'Goldman Sachs' in text_check_source :
    print('True')

True


In [59]:
# 檢查版本
clip_check_report = fitz.Rect(350, 0, rect.width, 250)
pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_check_report)
pix.save("page-clip_check_report.png")
text_check_report = page.get_text(clip=clip_check_report)
if 'Equities' in text_check_report:
    print('Equities')
else :
    print('NULL')
text_check_report

Equities


'th \nt \n8 March 2023 \n \nINITIATE AT REDUCE \nTARGET PRICE (TWD) \nPREVIOUS TARGET (TWD) \n \nEquities \nElectronic Equipment & \nInstruments \nTaiwan \n'

In [60]:
# 報告為新版個股報告 提取評價的第一種方法
try:
    clip_old_report_1 = fitz.Rect(375, 0, rect.width, 500)
    text_old_report_1 = page.get_text(clip=clip_old_report_1, sort=True).strip()
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_old_report_1)
    pix.save("page-clip_new_report_1.png")
    text_old_report_1 = text_old_report_1.split('TARGET')[0].strip()
    rate_1 = text_old_report_1.split('\n')[-1].strip()
except:
    rate_1 = None
rate_1

'INITIATE AT REDUCE'

In [61]:
# 報告為新版個股報告 提取評價的第二種方法
try:
    clip_old_report_2 = fitz.Rect(370, 190, rect.width, 230)
    rate_2 = page.get_text(clip=clip_old_report_2).strip()
    pix = page.get_pixmap(matrix=mat, alpha=False, clip=clip_old_report_2)
    pix.save("page-clip_new_report_2.png")
except:
    rate_2 = None
rate_2

'INITIATE AT REDUCE'